In [894]:
import pandas as pd
import numpy as np
import re

- "Lithium source", "Voltage range min", "Voltage range max", "Temperature" 변수 채점 자동화 처리 해야함!
- relevance rag 44번 논문 샘플명이 변형되어서 나옴
- 

In [895]:
hyper_method_name = "multiagent-rag_9"

In [896]:
gt = pd.read_csv("../output/csv/GT.csv", encoding="cp949")
answer = pd.read_csv(f"../output/csv/{hyper_method_name}.csv", encoding="cp949") ## .drop(columns="Additional treatment")

In [897]:
gt["Temperature"].apply(lambda x: 25 if x == "room temperature" else x)[0] 
answer["Temperature"].apply(lambda x: 25 if x == "room temperature" else x)[0]

nan

In [898]:
def text_processor(df): 
    ## `Voltage range` 변수 처리
    df["Voltage range"] = df["Voltage range"].apply(
        lambda x: x.replace("V", "").replace("v", "").replace("?", "-").replace("--", "-") if x is not np.NaN else None
    )
    
    df["Voltage range min"] = df["Voltage range"].apply(
        lambda x: float(x.split('-')[0]) if isinstance(x, str) and '-' in x else None
    )

    df["Voltage range max"] = df["Voltage range"].apply(
        lambda x: float(x.split('-')[1]) if isinstance(x, str) and '-' in x else None
    )

    df = df.drop(columns=["Voltage range"])


    ## `Temperature` 변수 처리
    df["Temperature"] = df["Temperature"].apply(lambda x: 25 if x == "room temperature" else x)
    
    return df   

In [899]:
answer["Voltage range"]

0       2.5-4.3
1       2.5-4.3
2       2.5-4.3
3       2.5-4.3
4     2.50-4.50
5     2.50-4.50
6       3.0-4.3
7       3.0-4.3
8       3.0-4.3
9       3.0-4.3
10      3.0-4.3
11      3.0-4.3
12      3.0-4.3
13      2.8-4.3
14      2.8-4.3
15      3.0?4.3
16    not found
17      2.6-4.2
18      2.6-4.2
19      2.6-4.2
20    2.75?4.30
21    2.75?4.30
22     3.0-4.55
23    not found
24      3.0-4.3
25      3.0-4.3
26          NaN
27          NaN
Name: Voltage range, dtype: object

In [900]:
gt = text_processor(gt)
answer = text_processor(answer)

In [901]:
Morphological_Properties_col = ['Particle size', 'Particle shape', 'Particle distribution', 'Coating layer characteristics', 'Crystal structure and lattice characteristics']

In [902]:
gt = gt.drop(columns=Morphological_Properties_col)
answer = answer.drop(columns=Morphological_Properties_col)

In [903]:
def c_rate_func(x):
    if "C-rate" in x:
        return True
    else: 
        return False

In [904]:
def combine_duplicated_cols(df):
    int_crate_cols = [
        col for col in df.columns
        if re.fullmatch(r"C-rate\s+[0-9]+", col)
    ]    
    
    int2float_map = {}
    for int_col in int_crate_cols:
        int2float_map[f"{int_col}"] = int_col[:7] + f"{float(int_col[7:])}"

    for int_col, float_col in int2float_map.items():
        if float_col in df.columns:
            df[int_col] = df[int_col].combine_first(df[float_col])
            df.drop(columns=float_col, inplace=True)
        df.rename(columns={int_col: float_col}, inplace=True)
    
    return df

In [905]:
gt = combine_duplicated_cols(gt)
answer = combine_duplicated_cols(answer)

In [906]:
gt_c_rate = gt.columns[gt.columns.map(c_rate_func)].to_list()
answer_c_rate = answer.columns[answer.columns.map(c_rate_func)].to_list()

In [907]:
other_c_rate_col = [
    'C-rate 0.05',
    'C-rate 5.0',
    'C-rate 10.0',
    'C-rate 20.0',
    'C-rate 40.0',
    'C-rate 0.33',
    'C-rate None',
    'C-rate 0.2',
    'C-rate 0.3',
    'C-rate 4.0',
    'C-rate 6.0',
]

In [908]:
paper_id = gt["Paper ID"].unique()

In [909]:
gt.head()

,Paper ID,Sample,Li ratio,Ni ratio,Co ratio,Mn ratio,O ratio,W ratio,Commercial NCM used,Lithium source,...,C-rate 2.0,C-rate 5.0,C-rate 10.0,C-rate 20.0,C-rate 40.0,C-rate None,C-rate 4.0,C-rate 6.0,Voltage range min,Voltage range max
0,11,NR0,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,45.5,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
1,11,NR1,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,47.0,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
2,11,NR3,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,75.4,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
3,11,NR5,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,NaN,95.4,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
4,16,NCM,1,0.33,0.33,0.33,2,NaN,no,LiAc,...,143.5,101.7,71.5,44.5,13.2,NaN,NaN,NaN,2.5,4.5


In [910]:
answer.head()

,Paper ID,Sample,Li ratio,Ni ratio,Co ratio,Mn ratio,O ratio,W ratio,Commercial NCM used,Lithium source,...,C-rate 5.0,C-rate 4.0,C-rate 10.0,C-rate 20.0,C-rate 40.0,C-rate 0.05,C-rate 6.0,C-rate None,Voltage range min,Voltage range max
0,11.0,NR0,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,45.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
1,11.0,NR1,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
2,11.0,NR3,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,75.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
3,11.0,NR5,1,0.33,0.33,0.33,2,NaN,no,LiNO3,...,95.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,4.3
4,16.0,NCM,1,0.33,0.33,0.33,NaN,NaN,no,LiAc,...,101.7,NaN,71.5,44.5,13.2,NaN,NaN,NaN,2.5,4.5


In [911]:
sort_col = [
    'Paper ID',
    'Sample',
    'Li ratio',
    'Ni ratio',
    'Co ratio',
    'Mn ratio',
    'O ratio',
    'W ratio',
    'Commercial NCM used',
    'Lithium source',
    'Synthesis method',
    'Crystallization method',
    'Crystallization final temperature',
    'Crystallization final duration (hours)',
    'Doping',
    'Coating',
    'Active material to Conductive additive to Binder ratio',
    'Electrolyte salt',
    'Electrolyte concentration',
    'Electrolyte solvent',
    'Electrolyte solvent ratio',
    'Additive',
    'Loading density (mass loading of NCM)',
    'Voltage range min',
    'Voltage range max',
    'Temperature',
    'C-rate 0.1',
    'C-rate 0.5',
    'C-rate 1.0',
    'C-rate 2.0',
    'C-rate 0.05',
    'C-rate 5.0',
    'C-rate 10.0',
    'C-rate 20.0',
    'C-rate 40.0',
    'C-rate 0.33',
    'C-rate None',
    'C-rate 0.2',
    'C-rate 0.3',
    'C-rate 4.0',
    'C-rate 6.0',
]

In [912]:
tf_col = [
    "Li ratio", 
    "Ni ratio", 
    "Co ratio", 
    "Mn ratio", 
    "O ratio", 
    "W ratio", 
    "Commercial NCM used",
    'Lithium source',
    "Crystallization final temperature", 
    "Crystallization final duration (hours)", 
    "Electrolyte salt", 
    'Voltage range min',
    'Voltage range max',
    'Temperature',
    'C-rate 0.1',
    'C-rate 0.5',
    'C-rate 1.0',
    'C-rate 2.0',    
    'C-rate 0.05',
    'C-rate 5.0',
    'C-rate 10.0',
    'C-rate 20.0',
    'C-rate 40.0',
    'C-rate 0.33',
    'C-rate None',
    'C-rate 0.2',
    'C-rate 0.3',
    'C-rate 4.0',
    'C-rate 6.0',
]

In [913]:
import pandas as pd

def is_number(val):
    try:
        float(val)
        return True
    except:
        return False

# key column 정의
key_cols = ["Paper ID", "Sample"]

# 모든 열 모음 (비교 대상)
all_cols = set(gt.columns) | set(answer.columns)
compare_cols = [col for col in all_cols if col not in key_cols]

# 결과 저장
results = []

# 모든 논문 처리
all_papers = sorted(set(gt["Paper ID"]) | set(answer["Paper ID"]))

for pid in all_papers:
    gt_group = gt[gt["Paper ID"] == pid].reset_index(drop=True)
    ans_group = answer[answer["Paper ID"] == pid].reset_index(drop=True)

    max_len = max(len(gt_group), len(ans_group))

    def pad_df(df, target_len):
        pad_len = target_len - len(df)
        if pad_len <= 0:
            return df
        empty = pd.DataFrame({col: [None] * pad_len for col in df.columns})
        empty["Paper ID"] = pid
        return pd.concat([df, empty], ignore_index=True)

    gt_group = pad_df(gt_group, max_len)
    ans_group = pad_df(ans_group, max_len)

    for i in range(max_len):
        row_gt = gt_group.iloc[i]
        row_ans = ans_group.iloc[i]

        result_row = {
            "Paper ID": pid,
            "Sample": row_gt["Sample"] if pd.notna(row_gt["Sample"]) else row_ans["Sample"]
        }

        for col in compare_cols:
            val_gt = row_gt[col] if col in row_gt and col in gt.columns else None
            val_ans = row_ans[col] if col in row_ans and col in answer.columns else None

            try:
                # C-rate 열이고 둘 다 NaN이면, 결과도 NaN
                if "C-rate" in col and pd.isna(val_gt) and pd.isna(val_ans):
                    match = float('nan')
                elif pd.isna(val_gt) and pd.isna(val_ans):
                    match = True
                elif pd.notna(val_gt) and pd.notna(val_ans):
                    if is_number(val_gt) and is_number(val_ans):
                        match = abs(float(val_gt) - float(val_ans)) <= 0.001
                        
                    else:
                        match = str(val_gt).strip() == str(val_ans).strip()
                else:
                    match = False

            except:
                match = False

            # 결과 저장 방식 변경
            if col in tf_col:
                result_row[col] = match
            else:
                result_row[col] = val_ans  # 숫자 컬럼이 아닌 경우는 answer 값 그대로 저장

        results.append(result_row)

# 결과 DataFrame
df_result = pd.DataFrame(results)

# 누락된 열 채우기 (혹시라도)
for col in tf_col:
    if col not in df_result.columns:
        df_result[col] = False

# 컬럼 순서 정렬
df_result = df_result[sort_col]

# 저장
df_result.to_csv(f"../output/csv/acc_{hyper_method_name}.csv", index=False)

In [914]:
gt[["Sample", "Lithium source", 'Voltage range min', 'Voltage range max', 'Temperature']]

,Sample,Lithium source,Voltage range min,Voltage range max,Temperature
0,NR0,LiNO3,2.50,4.30,25
1,NR1,LiNO3,2.50,4.30,25
2,NR3,LiNO3,2.50,4.30,25
3,NR5,LiNO3,2.50,4.30,25
4,NCM,LiAc,2.50,4.50,30
5,NCM/C,LiAc,2.50,4.50,30
6,Pristine,LiOH,3.00,4.30,25
7,V-0.005,LiOH,3.00,4.30,25
8,V-0.01,LiOH,3.00,4.30,25
9,V-0.02,LiOH,NaN,NaN,NaN


In [915]:
answer[["Sample", "Lithium source", 'Voltage range min', 'Voltage range max', 'Temperature']]

,Sample,Lithium source,Voltage range min,Voltage range max,Temperature
0,NR0,LiNO3,2.50,4.30,NaN
1,NR1,LiNO3,2.50,4.30,NaN
2,NR3,LiNO3,2.50,4.30,NaN
3,NR5,LiNO3,2.50,4.30,NaN
4,NCM,LiAc,2.50,4.50,30
5,NCM/C,LiAc,2.50,4.50,30
6,Pristine,LiOH,3.00,4.30,25
7,V-0.005,LiOH,3.00,4.30,25
8,V-0.01,LiOH,3.00,4.30,25
9,V-0.02,LiOH,3.00,4.30,25


In [916]:
df_result[["Sample", "Lithium source", 'Voltage range min', 'Voltage range max', 'Temperature']]

,Sample,Lithium source,Voltage range min,Voltage range max,Temperature
0,NR0,True,True,True,False
1,NR1,True,True,True,False
2,NR3,True,True,True,False
3,NR5,True,True,True,False
4,NCM,True,True,True,True
5,NCM/C,True,True,True,True
6,Pristine,True,True,True,True
7,V-0.005,True,True,True,True
8,V-0.01,True,True,True,True
9,V-0.02,True,False,False,False
